In [154]:
import gather_keys_oauth2 as Oauth2
import fitbit
import pandas as pd 
import datetime
import hashlib
import random
import json
from tqdm import tqdm
from datetime import datetime, timedelta

CLIENT_ID='23QSLM'
CLIENT_SECRET='8ed82541d286c938f9b8bc3f4e46da82'

In [155]:
server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

[01/May/2023:13:55:01] ENGINE Listening for SIGTERM.
[01/May/2023:13:55:01] ENGINE Listening for SIGHUP.
[01/May/2023:13:55:01] ENGINE Listening for SIGUSR1.
[01/May/2023:13:55:01] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[01/May/2023:13:55:01] ENGINE Started monitor thread 'Autoreloader'.
[01/May/2023:13:55:01] ENGINE Serving on http://127.0.0.1:8080
[01/May/2023:13:55:01] ENGINE Bus STARTED


127.0.0.1 - - [01/May/2023:13:55:03] "GET /?code=34b0e69b98171a74cc30bdd3e91b30fdf5a82226&state=sUkU7EOpvxFrD5ndiGqFawR7REGCBr HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15"


[01/May/2023:13:55:04] ENGINE Bus STOPPING
[01/May/2023:13:55:04] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[01/May/2023:13:55:04] ENGINE Stopped thread 'Autoreloader'.
[01/May/2023:13:55:04] ENGINE Bus STOPPED
[01/May/2023:13:55:04] ENGINE Bus EXITING
[01/May/2023:13:55:04] ENGINE Bus EXITED
[01/May/2023:13:55:04] ENGINE Waiting for child threads to terminate...


In [156]:
def generate_id():
    random_str = str(random.getrandbits(256)).encode('utf-8')
    sha2_hash = hashlib.sha256(random_str).hexdigest()
    return sha2_hash[:32]

### Create Heart Rate Data

In [ ]:
startTime = pd.to_datetime('2023-04-01',format='%Y-%m-%d')
endTime = pd.to_datetime('2023-04-30',format='%Y-%m-%d')

allDates = pd.date_range(start=startTime, end = endTime)

documents = []
heartRateZones_df_list = []
heartRate_df_list = []

for date in allDates:

    oneDayData = auth2_client.intraday_time_series('activities/heart',base_date=date,detail_level='15min')

    heartRateZones = oneDayData['activities-heart'][0]['value']['heartRateZones']

    for zone in heartRateZones:
        heartRateZones_dict = {
            'id' : generate_id(),
            'type' : 'heart-rate-zone',
            'data' : {
                'date' : date.date().strftime("%Y-%m-%d")
            }
        }

        for key, value in zone.items():
            heartRateZones_dict['data'][key] = value

        documents.append(heartRateZones_dict)

    heartRateZones_df = pd.DataFrame(heartRateZones)
    heartRateZones_df['date'] = date.date().strftime("%Y-%m-%d")
    heartRateZones_df_list.append(heartRateZones_df)

    heartIntraday = oneDayData['activities-heart-intraday']['dataset']

    for heartrate in heartIntraday:

        # date_obj = datetime.strptime(date, '%Y-%m-%d')
        time_obj = datetime.strptime(heartrate['time'], '%H:%M:%S').time()
        new_date_obj = datetime.combine(date, time_obj) + timedelta(hours=0)

        heartRate_dict = {
            'id' : generate_id(),
            'type' : 'heart-rate',
            'data' : {
                'datetime' : datetime.strftime(new_date_obj, '%Y-%m-%d %H:%M:%S'),
                'value' : heartrate['value']
            }
        }

        documents.append(heartRate_dict)

    heartRate_df = pd.DataFrame(heartIntraday)
    if heartRate_df.empty:
        continue
    heartRate_df['date'] = date.date().strftime("%Y-%m-%d")
    heartRate_df['datetime'] = pd.to_datetime(heartRate_df['date'] + ' ' + heartRate_df['time'])
    heartRate_df_list.append(heartRate_df)

json_str = json.dumps(documents)
with open(f'documents_heart_rate.json', 'w') as f:
    f.write(json_str)

heartRateZones_df_final = pd.concat(heartRateZones_df_list, axis = 0)
heartRateZones_df_final.to_csv('heartRateZones.csv',index=False)

heartRate_df_final = pd.concat(heartRate_df_list, axis = 0)
heartRate_df_final.to_csv('heartRate.csv',index=False)

### Create Activities Data

In [175]:
startTime = pd.to_datetime('2023-04-01',format='%Y-%m-%d')
endTime = pd.to_datetime('2023-04-30',format='%Y-%m-%d')

allDates = pd.date_range(start=startTime, end = endTime)

documents = []
activities_df_list = []

for date in allDates:

    oneDayData = auth2_client.activities(date=date)

    activities = oneDayData['activities']

    for activity in activities:

        activity_dict ={
            'id' : generate_id(),
            'type' : 'activity',
            'data' : activity
        }

        activity_dict['data']['date'] = date.date().strftime("%Y-%m-%d")
        documents.append(activity_dict)

        activities_df = pd.DataFrame(activity, index=[0])
        activities_df['date'] = date.date().strftime("%Y-%m-%d")
        activities_df_list.append(activities_df)

json_str = json.dumps(documents)
with open(f'documents_activities.json', 'w') as f:
    f.write(json_str)

activities_df_final = pd.concat(activities_df_list, axis = 0)
activities_df_final.to_csv('activities.csv',index=False)


### Create Distances and Activitis Summary Data

In [176]:
startTime = pd.to_datetime('2023-04-01',format='%Y-%m-%d')
endTime = pd.to_datetime('2023-04-30',format='%Y-%m-%d')

allDates = pd.date_range(start=startTime, end = endTime)


distances_df_list = []
activities_summary_df_list = []

documents = []
for date in allDates:

    oneDayData = auth2_client.activities(date=date)

    # Get distances
    distances = oneDayData['summary']['distances']

    for distance in distances:
        distance_dict = {
            'id' : generate_id(),
            'type' : 'distance',
            'data' : distance
        }

        distance_dict['data']['date'] = date.date().strftime("%Y-%m-%d")

        documents.append(distance_dict)

    distances_df = pd.DataFrame(distances)
    distances_df['date'] = date.date().strftime("%Y-%m-%d")
    distances_df_list.append(distances_df)

    dict_for_df = {}

    for data in ['steps','floors','elevation','lightlyActiveMinutes','fairlyActiveMinutes','veryActiveMinutes']:
        data_value = oneDayData['summary'][data]
        data_dict = {
                'id' : generate_id(),
                'type' : data,
                'data' : {
                    'date' : date.date().strftime("%Y-%m-%d"),
                    'value' : data_value
                }
            }
        documents.append(data_dict)

        dict_for_df[data] = { 'value' : data_value}

    activities_summary = pd.DataFrame(dict_for_df)
    activities_summary.reset_index(drop=True, inplace=True)
    activities_summary['date'] = date.date().strftime("%Y-%m-%d")
    activities_summary_df_list.append(activities_summary)

distances_df_final = pd.concat(distances_df_list, axis = 0)
activities_summary_df_final = pd.concat(activities_summary_df_list, axis = 0)

distances_df_final.to_csv('distances.csv',index=False)
activities_summary_df_final.to_csv('activities_summary.csv',index=False)

json_str = json.dumps(documents)
with open(f'documents_distances.json', 'w') as f:
    f.write(json_str)

In [219]:
heartRate = pd.read_csv('heartRate.csv')

heartRate_max = heartRate.groupby('date').agg({'value': 'max'})
heartRate_max = heartRate_max.rename(columns={'value':'Max'})
heartRate_max.reset_index(inplace=True)
heartRate_min = heartRate.groupby('date').agg({'value': 'min'})
heartRate_min = heartRate_min.rename(columns={'value':'Min'})
heartRate_min.reset_index(inplace=True,drop=True)
heartRate_mean = round(heartRate.groupby('date').agg({'value': 'mean'}),1)
heartRate_mean = heartRate_mean.rename(columns={'value':'Mean'})
heartRate_mean.reset_index(inplace=True)
df = pd.concat([heartRate_max,heartRate_min,heartRate_mean], axis=1)
df

,date,Max,date,Min,date,Mean
0,2023-04-01,112,2023-04-01,59,2023-04-01,75.0
1,2023-04-02,117,2023-04-02,51,2023-04-02,71.8
2,2023-04-03,99,2023-04-03,53,2023-04-03,70.1
3,2023-04-04,106,2023-04-04,50,2023-04-04,67.6
4,2023-04-05,105,2023-04-05,52,2023-04-05,73.9
5,2023-04-06,134,2023-04-06,53,2023-04-06,71.3
6,2023-04-10,99,2023-04-10,55,2023-04-10,79.4
7,2023-04-11,99,2023-04-11,53,2023-04-11,69.6
8,2023-04-12,94,2023-04-12,56,2023-04-12,68.9
9,2023-04-13,142,2023-04-13,55,2023-04-13,72.4
